# Agrupación de Direcciones por Cercanía usando Clustering Jerárquico

Este notebook realiza una agrupación de direcciones a partir de sus coordenadas geográficas (Latitud, Longitud) utilizando el algoritmo de **Agglomerative Clustering**. El objetivo es distribuir las direcciones en grupos de proximidad, con un **máximo de 15 direcciones por grupo**, manteniendo la coherencia espacial.

## Paso 1: Cargar los datos de direcciones con coordenadas

In [ ]:
import pandas as pd

# Cargar archivo con coordenadas previamente obtenidas
data = pd.read_excel('output_file_with_coordinates.xlsx')

## Paso 2: Escalado de coordenadas geográficas

In [ ]:
from sklearn.preprocessing import StandardScaler

# Escalar coordenadas para mejorar el agrupamiento
scaler = StandardScaler()
coords_scaled = scaler.fit_transform(data[['Latitud', 'Longitud']])

## Paso 3: Agrupamiento inicial con Agglomerative Clustering

In [ ]:
from sklearn.cluster import AgglomerativeClustering

max_grupos = 60
clustering = AgglomerativeClustering(n_clusters=max_grupos, linkage='ward')
data['Grupo_Temporal'] = clustering.fit_predict(coords_scaled)

## Paso 4: División de grupos con más de 15 direcciones por distancia al centro

In [ ]:
import numpy as np

centro_valledupar = [10.4631, -73.2532]
max_direcciones_por_grupo = 15
grupo_id = 0
final_grupos = []

for grupo in data['Grupo_Temporal'].unique():
    subgrupo = data[data['Grupo_Temporal'] == grupo].copy()
    subgrupo['DistanciaCentro'] = np.sqrt(
        (subgrupo['Latitud'] - centro_valledupar[0])**2 +
        (subgrupo['Longitud'] - centro_valledupar[1])**2
    )
    subgrupo = subgrupo.sort_values('DistanciaCentro')
    for i in range(0, len(subgrupo), max_direcciones_por_grupo):
        bloque = subgrupo.iloc[i:i + max_direcciones_por_grupo].copy()
        bloque['Grupo_Final'] = grupo_id
        final_grupos.append(bloque)
        grupo_id += 1

data_final = pd.concat(final_grupos)
data_final.drop(columns='DistanciaCentro', inplace=True)

## Detalle de la lógica de división para grupos grandes

Luego del agrupamiento inicial utilizando Agglomerative Clustering (paso 3), observamos que algunos grupos naturales contenían más de 15 direcciones. Sin embargo, como nuestro objetivo era mantener grupos operativos manejables (máximo 15 direcciones), decidimos dividir manualmente estos grupos grandes.

En lugar de forzar al algoritmo a generar un número exacto de grupos desde el principio —lo cual podría resultar en agrupaciones artificiales o poco coherentes geográficamente— aplicamos la siguiente lógica:

1. **Se conservaron los grupos generados naturalmente** por proximidad geográfica.
2. Para cada grupo con más de 15 direcciones:
   - Se calculó la **distancia de cada punto al centro de Valledupar**.
   - Se ordenaron las direcciones **de más cerca a más lejos** del centro.
   - Se dividieron en bloques de 15 direcciones, respetando esa secuencia geográfica.

Este enfoque permite preservar la **coherencia espacial** dentro de los subgrupos y evita crear agrupaciones artificiales que no representen proximidad real entre direcciones. Además, es más flexible y visualmente lógico al momento de asignar tareas operativas, rutas o coberturas zonales.

## Paso 5: Filtrado de grupos con menos de 3 direcciones

In [ ]:
# Filtrar grupos pequeños
conteo = data_final['Grupo_Final'].value_counts()
grupos_validos = conteo[conteo >= 3].index
df_filtrado = data_final[data_final['Grupo_Final'].isin(grupos_validos)].copy()

## Paso 6: Visualización de mapas por grupo

In [ ]:
import folium

def mostrar_mapa_por_grupo(df, grupo_col, grupo_id):
    grupo_df = df[df[grupo_col] == grupo_id]
    m = folium.Map(location=[10.4631, -73.2532], zoom_start=13)
    for _, row in grupo_df.iterrows():
        folium.CircleMarker(
            location=[row['Latitud'], row['Longitud']],
            radius=5,
            popup=f"Grupo: {row[grupo_col]}<br>Dirección: {row['DIRECCION']}",
            color='blue',
            fill=True,
            fill_color='blue'
        ).add_to(m)
    return m

# Ejemplo: mostrar grupo 5
mostrar_mapa_por_grupo(df_filtrado, 'Grupo_Final', 5)


## Visualización alternativa con Plotly

Esta visualización usa `plotly.express` para representar todos los grupos sobre un mapa interactivo. A diferencia de `folium`, este mapa **sí se puede renderizar correctamente dentro de notebooks vistos desde GitHub** o nbviewer.

Cada punto representa una dirección agrupada, coloreada por su grupo de proximidad (`Grupo_Final`).


In [ ]:

import plotly.express as px

# Visualización de agrupaciones usando Plotly
fig = px.scatter_mapbox(
    df_filtrado,
    lat="Latitud",
    lon="Longitud",
    color="Grupo_Final",
    hover_name="DIRECCION",
    zoom=12,
    mapbox_style="carto-positron",
    height=700,
    title="Direcciones agrupadas por proximidad geográfica"
)

fig.show()
